In [1]:
pip install matplotlib seaborn plotly

   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.9 MB 4.2 MB/s eta 0:00:03
   ------- -------------------------------- 1.8/9.9 MB 5.9 MB/s eta 0:00:02
   ------------ --------------------------- 3.1/9.9 MB 6.1 MB/s eta 0:00:02
   ------------------- -------------------- 4.7/9.9 MB 6.2 MB/s eta 0:00:01
   ------------------------ --------------- 6.0/9.9 MB 6.3 MB/s eta 0:00:01
   ----------------------------- ---------- 7.3/9.9 MB 6.3 MB/s eta 0:00:01
   ---------------------------------- ----- 8.7/9.9 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------  9.7/9.9 MB 6.2 MB/s eta 0:00:01
   ---------------------------------------  9.7/9.9 MB 6.2 MB/s eta 0:00:01
   ---------------------------------------- 9.9/9.9 MB 5.1 MB/s  0:00:02

   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwhals]
   ---------------------------------------- 0/2 [narwha

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import PercentFormatter
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

class VaccinationCoverageAnalyzer:
    def __init__(self, data_folder_path="Data files"):
        self.datafiles = data_folder_path
        self.all_regions = [
            "north east", "north west", "yorkshire and the humber",
            "east midlands", "west midlands", "east of england",
            "london", "south east", "south west"
        ]
        self.all_years = list(range(2006, 2020))
        self.res = pd.DataFrame()
    
    def load_data(self):
        """Load and prepare the main datasets from the specified folder - CSV ONLY"""
        data_path = Path(self.datafiles)
        
        if not data_path.exists():
            raise FileNotFoundError(f"Data folder '{self.datafiles}' not found. Please check the path.")
        
        print(f"Loading CSV data from '{self.datafiles}'...")
        
        # Load multiple outcomes CSV files with semicolon separator
        self.df = self._load_multiple_outcomes_files()
        
        # Load single files for study population and NHS data
        self.pop = self._load_single_file("study_population", sep=";")
        self.nhs = self._load_single_file("nhs_vacc", sep=",")
        
        print("✅ All CSV files loaded successfully")
        
        # Show column information
        self._show_column_info()
        
        self._process_data()
    
    def _show_column_info(self):
        """Show available columns in each dataset"""
        print("\n📊 Dataset Column Information:")
        print(f"Outcomes data: {len(self.df)} records, columns: {list(self.df.columns)}")
        print(f"Population data: {len(self.pop)} records, columns: {list(self.pop.columns)}")
        print(f"NHS data: {len(self.nhs)} records, columns: {list(self.nhs.columns)}")
    
    def _load_single_file(self, file_base_name, sep=";"):
        """Load a single CSV file with specified separator"""
        data_path = Path(self.datafiles)
        file_path = data_path / f"{file_base_name}.csv"
        
        if not file_path.exists():
            raise FileNotFoundError(f"Required file '{file_base_name}.csv' not found in {self.datafiles}")
        
        try:
            try:
                df = pd.read_csv(file_path, sep=sep)
            except:
                df = pd.read_csv(file_path, sep=",")
            
            print(f"✓ Loaded {file_path.name} ({len(df)} records)")
            return df
        except Exception as e:
            raise Exception(f"Error loading {file_path.name}: {e}")
    
    def _load_multiple_outcomes_files(self):
        """Load and combine multiple outcomes CSV files with semicolon separator"""
        data_path = Path(self.datafiles)
        outcomes_files = []
        
        patterns = [
            "outcomes_*.csv", "Outcomes_*.csv", "outcomes*.csv", 
            "Outcomes*.csv", "outcome_*.csv", "vaccination_*.csv",
            "mmr_*.csv", "outcomes.csv", "Outcomes.csv"
        ]
        
        for pattern in patterns:
            matching_files = list(data_path.glob(pattern))
            for file_path in matching_files:
                if file_path not in outcomes_files:
                    outcomes_files.append(file_path)
        
        if not outcomes_files:
            raise FileNotFoundError(f"No outcomes CSV files found in {self.datafiles}")
        
        print(f"Found {len(outcomes_files)} outcomes files:")
        
        all_outcomes = []
        for file_path in outcomes_files:
            try:
                try:
                    df_part = pd.read_csv(file_path, sep=";")
                except:
                    df_part = pd.read_csv(file_path, sep=",")
                
                print(f"  ✓ Loaded {file_path.name} ({len(df_part)} records)")
                all_outcomes.append(df_part)
                
            except Exception as e:
                print(f"  ✗ Failed to load {file_path.name}: {e}")
                continue
        
        if not all_outcomes:
            raise FileNotFoundError("No outcomes files could be loaded successfully")
        
        combined_df = pd.concat(all_outcomes, ignore_index=True)
        print(f"✅ Combined {len(all_outcomes)} outcomes files → {len(combined_df)} total records")
        
        return combined_df

    def _process_data(self):
        """Process the loaded CSV data"""
        print("Processing data...")
        self._clean_column_names()
        
        if 'Region' in self.nhs.columns:
            self.nhs['region'] = self.nhs['Region'].astype(str).str.lower()
        
        if 'region' in self.df.columns:
            self.df['region'] = self.df['region'].astype(str).str.lower()
        
        region_labels = {str(i+1): region for i, region in enumerate(self.all_regions)}
        if 'region' in self.df.columns:
            self.df['region'] = self.df['region'].map(region_labels).fillna(self.df['region'])
        
        self._process_population_data()
        print("✅ Data processing completed")
    
    def _clean_column_names(self):
        """Clean column names that have quotes and semicolons"""
        print("Cleaning column names...")
        
        if len(self.df.columns) == 1 and ';' in self.df.columns[0]:
            first_row = self.df.iloc[0, 0]
            if ';' in first_row:
                print("Reconstructing outcomes data from semicolon-separated format...")
                self.df = self._reconstruct_semicolon_data(self.df)
        
        if len(self.pop.columns) == 1 and ';' in self.pop.columns[0]:
            first_row = self.pop.iloc[0, 0]
            if ';' in first_row:
                print("Reconstructing population data from semicolon-separated format...")
                self.pop = self._reconstruct_semicolon_data(self.pop)
        
        self.df.columns = [col.replace('"', '') for col in self.df.columns]
        self.pop.columns = [col.replace('"', '') for col in self.pop.columns]
        self.nhs.columns = [col.replace('"', '') for col in self.nhs.columns]
    
    def _reconstruct_semicolon_data(self, df):
        """Reconstruct dataframe from semicolon-separated single column"""
        all_rows = []
        for idx, row in df.iterrows():
            values = row[0].split(';')
            values = [v.replace('"', '') for v in values]
            all_rows.append(values)
        
        if all_rows:
            new_df = pd.DataFrame(all_rows[1:], columns=all_rows[0])
            return new_df
        return df
    
    def _process_population_data(self):
        """Process population data to match our analysis needs"""
        print("Processing population data...")
        
        population_mapping = {
            'Name': 'region', 'Area': 'region', 'Year': 'yob',
            'age0': 'age0', 'age1': 'age1', 'age2': 'age2',
            'age3': 'age3', 'age4': 'age4', 'age5': 'age5'
        }
        
        for old_col, new_col in population_mapping.items():
            if old_col in self.pop.columns and new_col not in self.pop.columns:
                self.pop.rename(columns={old_col: new_col}, inplace=True)
        
        if 'region' in self.pop.columns:
            self.pop['region'] = self.pop['region'].astype(str).str.lower()
    
    def calculate_coverage(self):
        """Calculate vaccination coverage - simplified for your data format"""
        print("Calculating coverage rates...")
        
        if all(col in self.df.columns for col in ['region', 'vaccine', 'n_dose', 'year', 'cov1y', 'cov2y', 'cov3y', 'cov4y', 'cov5y']):
            print("✅ Outcomes files already contain coverage data - using directly")
            self.res = self.df.copy()
            
            for col in ['cov1y', 'cov2y', 'cov3y', 'cov4y', 'cov5y']:
                if col in self.res.columns:
                    self.res[col] = pd.to_numeric(self.res[col], errors='coerce')
            
            self.res = self.res[self.res['vaccine'] == 'MMR']
            print(f"✅ Coverage data prepared: {len(self.res)} MMR records")
            
            return self.res
        else:
            print("❌ Required columns not found in outcomes data")
            return pd.DataFrame()

    def create_visualizations(self, output_dir="Results"):
        """Create comprehensive visualizations of the coverage data"""
        print("\n📊 Creating visualizations...")
        
        if len(self.res) == 0:
            print("⚠ No data available for visualizations")
            return
        
        # Create visualizations directory
        viz_dir = Path(output_dir) / "visualizations"
        viz_dir.mkdir(exist_ok=True)
        
        # 1. Coverage Trends Over Time
        self._plot_coverage_trends(viz_dir)
        
        # 2. Regional Comparison
        self._plot_regional_comparison(viz_dir)
        
        # 3. Dose Comparison
        self._plot_dose_comparison(viz_dir)
        
        # 4. Age-specific Coverage
        self._plot_age_coverage(viz_dir)
        
        # 5. Interactive Plotly Charts
        self._create_interactive_charts(viz_dir)
        
        # 6. Coverage Heatmaps
        self._plot_coverage_heatmaps(viz_dir)
        
        print(f"✅ All visualizations saved to {viz_dir}")

    def _plot_coverage_trends(self, viz_dir):
        """Plot coverage trends over time by dose"""
        fig, axes = plt.subplots(2, 1, figsize=(14, 10))
        
        # Plot for dose 1
        dose1_data = self.res[self.res['n_dose'] == 1]
        if len(dose1_data) > 0:
            yearly_avg_d1 = dose1_data.groupby('year')[['cov1y', 'cov2y', 'cov3y', 'cov4y', 'cov5y']].mean()
            
            for age_col in ['cov1y', 'cov2y', 'cov3y', 'cov4y', 'cov5y']:
                if age_col in yearly_avg_d1.columns:
                    axes[0].plot(yearly_avg_d1.index, yearly_avg_d1[age_col], 
                               marker='o', linewidth=2, label=f'Age {age_col[3]}')
            
            axes[0].set_title('MMR Dose 1 Coverage Trends by Age', fontsize=14, fontweight='bold')
            axes[0].set_ylabel('Coverage Rate', fontsize=12)
            axes[0].legend(title='Age (years)')
            axes[0].grid(True, alpha=0.3)
            axes[0].yaxis.set_major_formatter(PercentFormatter(1.0))
        
        # Plot for dose 2
        dose2_data = self.res[self.res['n_dose'] == 2]
        if len(dose2_data) > 0:
            yearly_avg_d2 = dose2_data.groupby('year')[['cov1y', 'cov2y', 'cov3y', 'cov4y', 'cov5y']].mean()
            
            for age_col in ['cov1y', 'cov2y', 'cov3y', 'cov4y', 'cov5y']:
                if age_col in yearly_avg_d2.columns:
                    axes[1].plot(yearly_avg_d2.index, yearly_avg_d2[age_col], 
                               marker='s', linewidth=2, label=f'Age {age_col[3]}')
            
            axes[1].set_title('MMR Dose 2 Coverage Trends by Age', fontsize=14, fontweight='bold')
            axes[1].set_xlabel('Year', fontsize=12)
            axes[1].set_ylabel('Coverage Rate', fontsize=12)
            axes[1].legend(title='Age (years)')
            axes[1].grid(True, alpha=0.3)
            axes[1].yaxis.set_major_formatter(PercentFormatter(1.0))
        
        plt.tight_layout()
        plt.savefig(viz_dir / 'coverage_trends.png', dpi=300, bbox_inches='tight')
        plt.savefig(viz_dir / 'coverage_trends.pdf', bbox_inches='tight')
        plt.close()
        print("✓ Created coverage trends plot")

    def _plot_regional_comparison(self, viz_dir):
        """Plot regional comparison of coverage"""
        # Average coverage by region and dose
        regional_avg = self.res.groupby(['region', 'n_dose'])[['cov5y']].mean().reset_index()
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        # Dose 1 by region
        dose1_regional = regional_avg[regional_avg['n_dose'] == 1]
        if len(dose1_regional) > 0:
            bars1 = ax1.barh(dose1_regional['region'], dose1_regional['cov5y'], 
                           color='skyblue', alpha=0.7)
            ax1.set_title('MMR Dose 1 Coverage by Region (Age 5)', fontsize=14, fontweight='bold')
            ax1.set_xlabel('Coverage Rate', fontsize=12)
            ax1.xaxis.set_major_formatter(PercentFormatter(1.0))
            
            # Add value labels on bars
            for bar in bars1:
                width = bar.get_width()
                ax1.text(width + 0.01, bar.get_y() + bar.get_height()/2, 
                        f'{width:.1%}', ha='left', va='center', fontweight='bold')
        
        # Dose 2 by region
        dose2_regional = regional_avg[regional_avg['n_dose'] == 2]
        if len(dose2_regional) > 0:
            bars2 = ax2.barh(dose2_regional['region'], dose2_regional['cov5y'], 
                           color='lightcoral', alpha=0.7)
            ax2.set_title('MMR Dose 2 Coverage by Region (Age 5)', fontsize=14, fontweight='bold')
            ax2.set_xlabel('Coverage Rate', fontsize=12)
            ax2.xaxis.set_major_formatter(PercentFormatter(1.0))
            
            # Add value labels on bars
            for bar in bars2:
                width = bar.get_width()
                ax2.text(width + 0.01, bar.get_y() + bar.get_height()/2, 
                        f'{width:.1%}', ha='left', va='center', fontweight='bold')
        
        plt.tight_layout()
        plt.savefig(viz_dir / 'regional_comparison.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Created regional comparison plot")

    def _plot_dose_comparison(self, viz_dir):
        """Compare coverage between dose 1 and dose 2"""
        # Prepare data for comparison
        dose_comparison = self.res.groupby(['year', 'n_dose'])['cov5y'].mean().reset_index()
        dose_pivot = dose_comparison.pivot(index='year', columns='n_dose', values='cov5y')
        
        fig, ax = plt.subplots(figsize=(12, 6))
        
        if 1 in dose_pivot.columns and 2 in dose_pivot.columns:
            width = 0.35
            years = dose_pivot.index
            x = np.arange(len(years))
            
            ax.bar(x - width/2, dose_pivot[1], width, label='Dose 1', alpha=0.7, color='blue')
            ax.bar(x + width/2, dose_pivot[2], width, label='Dose 2', alpha=0.7, color='red')
            
            ax.set_xlabel('Year', fontsize=12)
            ax.set_ylabel('Coverage Rate (Age 5)', fontsize=12)
            ax.set_title('MMR Dose 1 vs Dose 2 Coverage Comparison', fontsize=14, fontweight='bold')
            ax.set_xticks(x)
            ax.set_xticklabels(years, rotation=45)
            ax.legend()
            ax.grid(True, alpha=0.3)
            ax.yaxis.set_major_formatter(PercentFormatter(1.0))
        
        plt.tight_layout()
        plt.savefig(viz_dir / 'dose_comparison.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Created dose comparison plot")

    def _plot_age_coverage(self, viz_dir):
        """Plot coverage by age for different years"""
        # Get latest year data
        latest_year = self.res['year'].max()
        latest_data = self.res[self.res['year'] == latest_year]
        
        if len(latest_data) == 0:
            return
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        # Dose 1 age coverage
        dose1_latest = latest_data[latest_data['n_dose'] == 1]
        if len(dose1_latest) > 0:
            age_coverage_d1 = dose1_latest[['cov1y', 'cov2y', 'cov3y', 'cov4y', 'cov5y']].mean()
            ages = [1, 2, 3, 4, 5]
            ax1.plot(ages, age_coverage_d1.values, marker='o', linewidth=2, markersize=8, color='blue')
            ax1.set_title(f'MMR Dose 1 Coverage by Age ({latest_year})', fontsize=14, fontweight='bold')
            ax1.set_xlabel('Age (years)', fontsize=12)
            ax1.set_ylabel('Coverage Rate', fontsize=12)
            ax1.grid(True, alpha=0.3)
            ax1.yaxis.set_major_formatter(PercentFormatter(1.0))
            
            # Add value annotations
            for i, (age, cov) in enumerate(zip(ages, age_coverage_d1.values)):
                ax1.annotate(f'{cov:.1%}', (age, cov), textcoords="offset points", 
                           xytext=(0,10), ha='center', fontweight='bold')
        
        # Dose 2 age coverage
        dose2_latest = latest_data[latest_data['n_dose'] == 2]
        if len(dose2_latest) > 0:
            age_coverage_d2 = dose2_latest[['cov1y', 'cov2y', 'cov3y', 'cov4y', 'cov5y']].mean()
            ages = [1, 2, 3, 4, 5]
            ax2.plot(ages, age_coverage_d2.values, marker='s', linewidth=2, markersize=8, color='red')
            ax2.set_title(f'MMR Dose 2 Coverage by Age ({latest_year})', fontsize=14, fontweight='bold')
            ax2.set_xlabel('Age (years)', fontsize=12)
            ax2.set_ylabel('Coverage Rate', fontsize=12)
            ax2.grid(True, alpha=0.3)
            ax2.yaxis.set_major_formatter(PercentFormatter(1.0))
            
            # Add value annotations
            for i, (age, cov) in enumerate(zip(ages, age_coverage_d2.values)):
                ax2.annotate(f'{cov:.1%}', (age, cov), textcoords="offset points", 
                           xytext=(0,10), ha='center', fontweight='bold')
        
        plt.tight_layout()
        plt.savefig(viz_dir / 'age_coverage.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Created age coverage plot")

    def _create_interactive_charts(self, viz_dir):
        """Create interactive Plotly charts"""
        try:
            # Interactive line chart - coverage trends
            yearly_avg = self.res.groupby(['year', 'n_dose'])[['cov1y', 'cov2y', 'cov3y', 'cov4y', 'cov5y']].mean().reset_index()
            
            fig = px.line(yearly_avg, x='year', y='cov5y', color='n_dose',
                         title='Interactive MMR Coverage Trends by Dose',
                         labels={'cov5y': 'Coverage at Age 5', 'year': 'Year', 'n_dose': 'Dose'},
                         color_discrete_map={1: 'blue', 2: 'red'})
            
            fig.update_layout(
                hovermode='x unified',
                showlegend=True,
                yaxis_tickformat='.0%'
            )
            
            fig.write_html(viz_dir / "interactive_coverage_trends.html")
            
            # Regional heatmap
            regional_data = self.res.groupby(['region', 'n_dose'])['cov5y'].mean().reset_index()
            regional_pivot = regional_data.pivot(index='region', columns='n_dose', values='cov5y')
            
            fig2 = px.imshow(regional_pivot,
                           title='Regional MMR Coverage Heatmap',
                           color_continuous_scale='Blues',
                           aspect="auto")
            
            fig2.update_layout(
                xaxis_title="Dose",
                yaxis_title="Region"
            )
            
            fig2.write_html(viz_dir / "regional_heatmap.html")
            print("✓ Created interactive charts")
            
        except Exception as e:
            print(f"⚠ Could not create interactive charts: {e}")

    def _plot_coverage_heatmaps(self, viz_dir):
        """Create coverage heatmaps"""
        # Prepare data for heatmaps
        heatmap_data = self.res.groupby(['year', 'region'])['cov5y'].mean().unstack()
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))
        
        # Heatmap for all data
        if not heatmap_data.empty:
            sns.heatmap(heatmap_data, annot=True, fmt='.1%', cmap='YlGnBu', 
                       cbar_kws={'format': PercentFormatter(1.0)}, ax=ax1)
            ax1.set_title('MMR Coverage Heatmap by Year and Region', fontsize=14, fontweight='bold')
            ax1.set_xlabel('Region', fontsize=12)
            ax1.set_ylabel('Year', fontsize=12)
        
        # Latest year regional comparison
        latest_year = self.res['year'].max()
        latest_regional = self.res[self.res['year'] == latest_year].groupby('region')['cov5y'].mean().sort_values()
        
        if len(latest_regional) > 0:
            colors = plt.cm.RdYlGn(latest_regional.values)
            bars = ax2.barh(range(len(latest_regional)), latest_regional.values, color=colors)
            ax2.set_yticks(range(len(latest_regional)))
            ax2.set_yticklabels(latest_regional.index)
            ax2.set_title(f'Regional MMR Coverage - {latest_year}', fontsize=14, fontweight='bold')
            ax2.set_xlabel('Coverage Rate', fontsize=12)
            ax2.xaxis.set_major_formatter(PercentFormatter(1.0))
            
            # Add value labels
            for i, bar in enumerate(bars):
                width = bar.get_width()
                ax2.text(width + 0.01, bar.get_y() + bar.get_height()/2, 
                        f'{width:.1%}', ha='left', va='center', fontweight='bold')
        
        plt.tight_layout()
        plt.savefig(viz_dir / 'coverage_heatmaps.png', dpi=300, bbox_inches='tight')
        plt.close()
        print("✓ Created coverage heatmaps")

    def save_results(self, output_dir="Results"):
        """Save all results to CSV files"""
        Path(output_dir).mkdir(exist_ok=True)
        
        if len(self.res) > 0:
            self.res.to_csv(
                Path(output_dir) / "coverage_results.csv",
                sep=",", index=False
            )
            print(f"✅ Results saved to {output_dir}/coverage_results.csv")
            
            # Save summary statistics
            self._save_summary_statistics(output_dir)
            
            # Create visualizations
            self.create_visualizations(output_dir)
        else:
            print("⚠ No results to save")
    
    def _save_summary_statistics(self, output_dir):
        """Save summary statistics of the coverage data"""
        if len(self.res) == 0:
            return
        
        print("\n📈 Coverage Summary Statistics:")
        
        summary = self.res.groupby(['n_dose', 'region']).agg({
            'cov1y': 'mean',
            'cov2y': 'mean', 
            'cov3y': 'mean',
            'cov4y': 'mean',
            'cov5y': 'mean'
        }).round(4)
        
        print("Average coverage by dose and region:")
        print(summary)
        
        summary.to_csv(Path(output_dir) / "coverage_summary.csv")
        print(f"✅ Summary saved to {output_dir}/coverage_summary.csv")

def clean_path_input(user_input):
    """Clean and validate user path input"""
    cleaned = user_input.strip().strip('"').strip("'").strip()
    if cleaned.startswith('r'):
        cleaned = cleaned[1:].strip().strip('"').strip("'")
    return cleaned

# Main execution
def main():
    print("=== MMR Vaccination Coverage Analysis ===")
    
    try:
        # Get folder path from user
        user_input = input("Enter the path to your data folder (or press Enter for 'Data files'): ").strip()
        
        if not user_input:
            data_folder = "Data files"
        else:
            data_folder = clean_path_input(user_input)
        
        print(f"Looking for data in: {data_folder}")
        
        # Initialize analyzer with specified path
        analyzer = VaccinationCoverageAnalyzer(data_folder_path=data_folder)
        
        # Load data from specified folder
        analyzer.load_data()
        
        # Calculate coverage
        coverage_results = analyzer.calculate_coverage()
        
        # Save results and create visualizations
        analyzer.save_results()
        
        print("=== Analysis Complete ===")
        
    except FileNotFoundError as e:
        print(f"❌ ERROR: {e}")
        print("\nPlease check:")
        print("1. The folder path is correct")
        print("2. The folder contains the required CSV files:")
        print("   - outcomes_*.csv (multiple files)")
        print("   - study_population.csv")
        print("   - nhs_vacc.csv")
        print(f"\nCurrent working directory: {os.getcwd()}")
        sys.exit(1)
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        import traceback
        traceback.print_exc()
        sys.exit(1)

if __name__ == "__main__":
    main()

=== MMR Vaccination Coverage Analysis ===


Enter the path to your data folder (or press Enter for 'Data files'):  "C:\Users\GULLYHUB\Desktop\Data files"


Looking for data in: C:\Users\GULLYHUB\Desktop\Data files
Loading CSV data from 'C:\Users\GULLYHUB\Desktop\Data files'...
Found 8 outcomes files:
  ✓ Loaded Outcomes_earlyMMR2 (1).csv (270 records)
  ✓ Loaded Outcomes_earlyMMR2.csv (270 records)
  ✓ Loaded Outcomes_earlyMMR2plus1.csv (270 records)
  ✓ Loaded Outcomes_earlyMMR2plus3.csv (270 records)
  ✓ Loaded Outcomes_earlyMMR2_likeMMR1 (1).csv (270 records)
  ✓ Loaded Outcomes_earlyMMR2_likeMMR1.csv (270 records)
  ✓ Loaded Outcomes_earlyMMR2_minus5 (1).csv (270 records)
  ✓ Loaded outcomes_earlyMMR2_minus5.csv (270 records)
✅ Combined 8 outcomes files → 2160 total records
✓ Loaded study_population.csv (27 records)
✓ Loaded nhs_vacc.csv (310 records)
✅ All CSV files loaded successfully

📊 Dataset Column Information:
Outcomes data: 2160 records, columns: ['region', 'vaccine', 'n_dose', 'year', 'cov1y', 'cov2y', 'cov3y', 'cov4y', 'cov5y']
Population data: 27 records, columns: ['Unnamed: 0', 'Name', 'Area', 'Year', 'age0', 'age1', 'age2

In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
import sys
import os
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import PercentFormatter
import scipy.stats as stats
from scipy.optimize import curve_fit
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

class MMREpidemiologicalModel:
    def __init__(self, data_folder_path="Data files"):
        self.datafiles = data_folder_path
        self.all_regions = [
            "north east", "north west", "yorkshire and the humber",
            "east midlands", "west midlands", "east of england",
            "london", "south east", "south west"
        ]
        self.res = pd.DataFrame()
        
        # Epidemiological parameters for MMR (from literature)
        self.r0_mmr = 12  # Basic reproduction number for measles
        self.r0_mumps = 10  # Basic reproduction number for mumps  
        self.r0_rubella = 6  # Basic reproduction number for rubella
        self.vaccine_efficacy = 0.95  # 95% efficacy for 2 doses
        self.herd_immunity_threshold_mmr = 1 - (1/self.r0_mmr)  # ~92%
        self.herd_immunity_threshold_mumps = 1 - (1/self.r0_mumps)  # ~90%
        self.herd_immunity_threshold_rubella = 1 - (1/self.r0_rubella)  # ~83%
    
    def load_data(self):
        """Load and process the coverage data"""
        data_path = Path(self.datafiles)
        
        if not data_path.exists():
            raise FileNotFoundError(f"Data folder '{self.datafiles}' not found.")
        
        print(f"Loading CSV data from '{self.datafiles}'...")
        
        # Load outcomes data
        self.df = self._load_multiple_outcomes_files()
        self.res = self.df.copy()
        
        # Convert coverage to numeric
        for col in ['cov1y', 'cov2y', 'cov3y', 'cov4y', 'cov5y']:
            if col in self.res.columns:
                self.res[col] = pd.to_numeric(self.res[col], errors='coerce')
        
        self.res = self.res[self.res['vaccine'] == 'MMR']
        print(f"✅ Loaded {len(self.res)} MMR coverage records")
        
    def _load_multiple_outcomes_files(self):
        """Load outcomes CSV files"""
        data_path = Path(self.datafiles)
        outcomes_files = []
        
        patterns = ["Outcomes_*.csv", "outcomes_*.csv", "outcomes.csv"]
        
        for pattern in patterns:
            matching_files = list(data_path.glob(pattern))
            outcomes_files.extend(matching_files)
        
        all_outcomes = []
        for file_path in outcomes_files:
            try:
                df_part = pd.read_csv(file_path, sep=";")
                all_outcomes.append(df_part)
            except:
                try:
                    df_part = pd.read_csv(file_path, sep=",")
                    all_outcomes.append(df_part)
                except:
                    continue
        
        return pd.concat(all_outcomes, ignore_index=True)
    
    def calculate_herd_immunity_gap(self):
        """Calculate the gap to herd immunity threshold"""
        print("\n🛡️ HERD IMMUNITY ANALYSIS")
        print("=" * 50)
        
        # Get latest year data
        latest_year = self.res['year'].max()
        latest_data = self.res[self.res['year'] == latest_year]
        
        if len(latest_data) == 0:
            print("No data available for analysis")
            return
        
        # Calculate effective coverage (accounting for vaccine efficacy)
        dose2_coverage = latest_data[latest_data['n_dose'] == 2]['cov5y'].mean()
        effective_coverage = dose2_coverage * self.vaccine_efficacy
        
        print(f"Latest Year: {latest_year}")
        print(f"Average Dose 2 Coverage: {dose2_coverage:.1%}")
        print(f"Effective Coverage (95% efficacy): {effective_coverage:.1%}")
        print(f"\nHerd Immunity Thresholds:")
        print(f"Measles (R0={self.r0_mmr}): {self.herd_immunity_threshold_mmr:.1%}")
        print(f"Mumps (R0={self.r0_mumps}): {self.herd_immunity_threshold_mumps:.1%}")
        print(f"Rubella (R0={self.r0_rubella}): {self.herd_immunity_threshold_rubella:.1%}")
        
        # Calculate gaps
        measles_gap = max(0, self.herd_immunity_threshold_mmr - effective_coverage)
        mumps_gap = max(0, self.herd_immunity_threshold_mumps - effective_coverage)
        rubella_gap = max(0, self.herd_immunity_threshold_rubella - effective_coverage)
        
        print(f"\n📊 Coverage Gaps to Herd Immunity:")
        print(f"Measles: {measles_gap:.1%}")
        print(f"Mumps: {mumps_gap:.1%}")
        print(f"Rubella: {rubella_gap:.1%}")
        
        return {
            'effective_coverage': effective_coverage,
            'gaps': {
                'measles': measles_gap,
                'mumps': mumps_gap,
                'rubella': rubella_gap
            }
        }
    
    def outbreak_risk_assessment(self):
        """Assess outbreak risk based on coverage levels"""
        print("\n⚠️ OUTBREAK RISK ASSESSMENT")
        print("=" * 50)
        
        latest_year = self.res['year'].max()
        regional_data = self.res[
            (self.res['year'] == latest_year) & 
            (self.res['n_dose'] == 2)
        ]
        
        risk_categories = {
            'Very High Risk': 0.85,  # Below 85%
            'High Risk': 0.90,       # 85-90%
            'Moderate Risk': 0.92,   # 90-92%
            'Low Risk': 1.0          # Above 92%
        }
        
        regional_risks = []
        
        for region in self.all_regions:
            region_data = regional_data[regional_data['region'] == region]
            if len(region_data) > 0:
                coverage = region_data['cov5y'].mean()
                effective_coverage = coverage * self.vaccine_efficacy
                
                # Determine risk level
                risk_level = "Low Risk"
                for risk, threshold in risk_categories.items():
                    if effective_coverage < threshold:
                        risk_level = risk
                        break
                
                regional_risks.append({
                    'region': region,
                    'coverage': coverage,
                    'effective_coverage': effective_coverage,
                    'risk_level': risk_level,
                    'measles_susceptible': max(0, 1 - effective_coverage/self.herd_immunity_threshold_mmr)
                })
        
        risk_df = pd.DataFrame(regional_risks)
        
        print("Regional Outbreak Risk Assessment:")
        print(risk_df.to_string(index=False))
        
        return risk_df
    
    def policy_scenario_analysis(self):
        """Analyze different policy intervention scenarios"""
        print("\n📋 POLICY SCENARIO ANALYSIS")
        print("=" * 50)
        
        baseline_coverage = self.res[
            (self.res['n_dose'] == 2) & 
            (self.res['year'] == self.res['year'].max())
        ]['cov5y'].mean()
        
        scenarios = {
            'Baseline': baseline_coverage,
            'Improve Access (+5%)': min(1.0, baseline_coverage + 0.05),
            'School Entry Requirements (+8%)': min(1.0, baseline_coverage + 0.08),
            'Catch-up Campaigns (+10%)': min(1.0, baseline_coverage + 0.10),
            'Mandatory Vaccination (+15%)': min(1.0, baseline_coverage + 0.15),
            'Opt-out System (+12%)': min(1.0, baseline_coverage + 0.12)
        }
        
        scenario_results = []
        
        for scenario, coverage in scenarios.items():
            effective_coverage = coverage * self.vaccine_efficacy
            
            # Calculate outbreak probability (simplified model)
            outbreak_prob_measles = max(0, 1 - (effective_coverage / self.herd_immunity_threshold_mmr))
            outbreak_prob_mumps = max(0, 1 - (effective_coverage / self.herd_immunity_threshold_mumps))
            
            # Estimate cases prevented (simplified model)
            baseline_susceptible = max(0, 1 - (scenarios['Baseline'] * self.vaccine_efficacy / self.herd_immunity_threshold_mmr))
            scenario_susceptible = max(0, 1 - (effective_coverage / self.herd_immunity_threshold_mmr))
            cases_prevented_pct = (baseline_susceptible - scenario_susceptible) * 100
            
            scenario_results.append({
                'Scenario': scenario,
                'Coverage': coverage,
                'Effective_Coverage': effective_coverage,
                'Outbreak_Risk_Measles': outbreak_prob_measles,
                'Cases_Prevented_Pct': cases_prevented_pct,
                'Herd_Immunity_Measles': effective_coverage >= self.herd_immunity_threshold_mmr,
                'Herd_Immunity_Mumps': effective_coverage >= self.herd_immunity_threshold_mumps
            })
        
        results_df = pd.DataFrame(scenario_results)
        print("Policy Scenario Impacts:")
        print(results_df.round(3).to_string(index=False))
        
        return results_df
    
    def cost_benefit_analysis(self, population_size=1000000):
        """Simple cost-benefit analysis of vaccination programs"""
        print("\n💰 COST-BENEFIT ANALYSIS")
        print("=" * 50)
        
        # Cost assumptions (in GBP)
        cost_per_dose = 15
        cost_per_measles_case = 5000  # Includes healthcare and productivity loss
        cost_per_mumps_case = 3000
        cost_per_rubella_case = 4000
        
        # Disease incidence without vaccination
        annual_cases_per_100k = {
            'measles': 500,
            'mumps': 300,
            'rubella': 200
        }
        
        baseline_coverage = self.res[
            (self.res['n_dose'] == 2) & 
            (self.res['year'] == self.res['year'].max())
        ]['cov5y'].mean()
        
        effective_baseline = baseline_coverage * self.vaccine_efficacy
        
        # Calculate expected cases
        expected_cases = {}
        for disease, cases in annual_cases_per_100k.items():
            if disease == 'measles':
                threshold = self.herd_immunity_threshold_mmr
            elif disease == 'mumps':
                threshold = self.herd_immunity_threshold_mumps
            else:
                threshold = self.herd_immunity_threshold_rubella
            
            outbreak_prob = max(0, 1 - (effective_baseline / threshold))
            expected_cases[disease] = cases * outbreak_prob * (population_size / 100000)
        
        # Calculate costs
        vaccination_cost = population_size * 2 * cost_per_dose  # 2 doses per person
        disease_cost = sum(
            expected_cases[disease] * cost 
            for disease, cost in zip(['measles', 'mumps', 'rubella'], 
                                   [cost_per_measles_case, cost_per_mumps_case, cost_per_rubella_case])
        )
        
        total_expected_cases = sum(expected_cases.values())
        
        print(f"Population: {population_size:,}")
        print(f"Baseline Coverage: {baseline_coverage:.1%}")
        print(f"Effective Coverage: {effective_baseline:.1%}")
        print(f"\nExpected Annual Cases:")
        for disease, cases in expected_cases.items():
            print(f"  {disease.title()}: {cases:,.0f} cases")
        print(f"Total Expected Cases: {total_expected_cases:,.0f}")
        
        print(f"\nCost Analysis:")
        print(f"Vaccination Program Cost: £{vaccination_cost:,.0f}")
        print(f"Expected Disease Costs: £{disease_cost:,.0f}")
        print(f"Cost per Case Prevented: £{vaccination_cost/max(1, total_expected_cases):,.0f}")
        
        # Cost-benefit ratio (simplified)
        benefit_cost_ratio = disease_cost / vaccination_cost if vaccination_cost > 0 else 0
        print(f"Benefit-Cost Ratio: {benefit_cost_ratio:.2f}")
        
        return {
            'vaccination_cost': vaccination_cost,
            'disease_cost': disease_cost,
            'expected_cases': expected_cases,
            'benefit_cost_ratio': benefit_cost_ratio
        }
    
    def vulnerable_population_analysis(self):
        """Identify vulnerable subpopulations and geographic hotspots"""
        print("\n🎯 VULNERABLE POPULATION ANALYSIS")
        print("=" * 50)
        
        latest_year = self.res['year'].max()
        regional_data = self.res[
            (self.res['year'] == latest_year) & 
            (self.res['n_dose'] == 2)
        ]
        
        # Calculate vulnerability scores
        vulnerability_data = []
        
        for region in self.all_regions:
            region_data = regional_data[regional_data['region'] == region]
            if len(region_data) > 0:
                coverage = region_data['cov5y'].mean()
                effective_coverage = coverage * self.vaccine_efficacy
                
                # Vulnerability score (0-100, higher = more vulnerable)
                measles_gap = max(0, self.herd_immunity_threshold_mmr - effective_coverage)
                vulnerability_score = int(measles_gap * 100)
                
                # Priority level
                if vulnerability_score >= 15:
                    priority = "CRITICAL"
                elif vulnerability_score >= 10:
                    priority = "HIGH"
                elif vulnerability_score >= 5:
                    priority = "MEDIUM"
                else:
                    priority = "LOW"
                
                vulnerability_data.append({
                    'Region': region,
                    'Coverage': coverage,
                    'Effective_Coverage': effective_coverage,
                    'Measles_Gap': measles_gap,
                    'Vulnerability_Score': vulnerability_score,
                    'Priority_Level': priority
                })
        
        vulnerability_df = pd.DataFrame(vulnerability_data).sort_values('Vulnerability_Score', ascending=False)
        
        print("Regional Vulnerability Assessment:")
        print(vulnerability_df.to_string(index=False))
        
        return vulnerability_df
    
    def predictive_modelling(self, target_year=2025):
        """Predict future coverage and outbreak risks"""
        print(f"\n🔮 PREDICTIVE MODELLING ({target_year})")
        print("=" * 50)
        
        # Get historical trends
        historical = self.res[
            (self.res['n_dose'] == 2) & 
            (self.res['year'] >= 2010)
        ].groupby('year')['cov5y'].mean()
        
        if len(historical) < 3:
            print("Insufficient historical data for prediction")
            return
        
        # Simple linear projection
        years = np.array(historical.index)
        coverage = historical.values
        
        # Fit linear trend
        slope, intercept = np.polyfit(years, coverage, 1)
        
        # Predict future coverage
        future_years = np.array([2023, 2024, 2025])
        predicted_coverage = slope * future_years + intercept
        predicted_coverage = np.clip(predicted_coverage, 0, 1)  # Keep between 0-100%
        
        # Calculate future risks
        future_effective = predicted_coverage * self.vaccine_efficacy
        outbreak_risk = np.maximum(0, 1 - future_effective / self.herd_immunity_threshold_mmr)
        
        print("Future Coverage Predictions:")
        for year, cov, risk in zip(future_years, predicted_coverage, outbreak_risk):
            herd_immunity_achieved = cov * self.vaccine_efficacy >= self.herd_immunity_threshold_mmr
            status = "✅ HERD IMMUNITY" if herd_immunity_achieved else "❌ AT RISK"
            print(f"{year}: {cov:.1%} coverage | Outbreak risk: {risk:.1%} | {status}")
        
        return {
            'years': future_years,
            'predicted_coverage': predicted_coverage,
            'outbreak_risk': outbreak_risk
        }
    
    def create_epidemiological_dashboard(self, output_dir="Epidemiological_Models"):
        """Create comprehensive epidemiological dashboard"""
        output_path = Path(output_dir)
        output_path.mkdir(exist_ok=True)
        
        print(f"\n📈 CREATING EPIDEMIOLOGICAL DASHBOARD")
        print("=" * 50)
        
        # Run all analyses
        herd_immunity = self.calculate_herd_immunity_gap()
        risk_assessment = self.outbreak_risk_assessment()
        policy_scenarios = self.policy_scenario_analysis()
        cost_benefit = self.cost_benefit_analysis()
        vulnerability = self.vulnerable_population_analysis()
        predictions = self.predictive_modelling()
        
        # Create visualizations
        self._create_epidemiological_plots(output_path, herd_immunity, risk_assessment, 
                                         policy_scenarios, vulnerability, predictions)
        
        # Save results
        self._save_epidemiological_results(output_path, herd_immunity, risk_assessment,
                                         policy_scenarios, cost_benefit, vulnerability, predictions)
        
        print(f"✅ Epidemiological dashboard saved to {output_path}")
    
    def _create_epidemiological_plots(self, output_path, herd_immunity, risk_assessment,
                                    policy_scenarios, vulnerability, predictions):
        """Create epidemiological visualization plots"""
        
        # 1. Herd Immunity Gap Plot
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))
        
        # Herd immunity gaps
        if herd_immunity:
            diseases = ['Measles', 'Mumps', 'Rubella']
            gaps = [herd_immunity['gaps']['measles'], 
                   herd_immunity['gaps']['mumps'], 
                   herd_immunity['gaps']['rubella']]
            
            colors = ['red' if gap > 0 else 'green' for gap in gaps]
            bars = ax1.bar(diseases, gaps, color=colors, alpha=0.7)
            ax1.set_title('Gap to Herd Immunity Threshold', fontsize=14, fontweight='bold')
            ax1.set_ylabel('Coverage Gap', fontsize=12)
            ax1.yaxis.set_major_formatter(PercentFormatter(1.0))
            
            for bar, gap in zip(bars, gaps):
                ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                        f'{gap:.1%}', ha='center', va='bottom', fontweight='bold')
        
        # 2. Risk Assessment Map
        if not risk_assessment.empty:
            risk_colors = {'Very High Risk': 'red', 'High Risk': 'orange', 
                          'Moderate Risk': 'yellow', 'Low Risk': 'green'}
            risk_assessment['color'] = risk_assessment['risk_level'].map(risk_colors)
            
            ax2.barh(risk_assessment['region'], risk_assessment['effective_coverage'],
                    color=risk_assessment['color'], alpha=0.7)
            ax2.axvline(x=self.herd_immunity_threshold_mmr, color='red', linestyle='--', 
                       label=f'Measles Herd Immunity ({self.herd_immunity_threshold_mmr:.1%})')
            ax2.set_title('Regional Outbreak Risk Assessment', fontsize=14, fontweight='bold')
            ax2.set_xlabel('Effective Coverage', fontsize=12)
            ax2.xaxis.set_major_formatter(PercentFormatter(1.0))
            ax2.legend()
        
        # 3. Policy Scenario Impacts
        if not policy_scenarios.empty:
            scenarios = policy_scenarios['Scenario']
            cases_prevented = policy_scenarios['Cases_Prevented_Pct']
            
            ax3.barh(scenarios, cases_prevented, color='steelblue', alpha=0.7)
            ax3.set_title('Policy Impact: Cases Prevented (%)', fontsize=14, fontweight='bold')
            ax3.set_xlabel('Percentage Reduction in Cases', fontsize=12)
            
            for i, v in enumerate(cases_prevented):
                ax3.text(v + 0.5, i, f'{v:.1f}%', va='center', fontweight='bold')
        
        # 4. Vulnerability Analysis
        if not vulnerability.empty:
            ax4.barh(vulnerability['Region'], vulnerability['Vulnerability_Score'],
                    color='coral', alpha=0.7)
            ax4.set_title('Regional Vulnerability Scores', fontsize=14, fontweight='bold')
            ax4.set_xlabel('Vulnerability Score (0-100)', fontsize=12)
            
            for i, v in enumerate(vulnerability['Vulnerability_Score']):
                ax4.text(v + 1, i, str(v), va='center', fontweight='bold')
        
        plt.tight_layout()
        plt.savefig(output_path / 'epidemiological_dashboard.png', dpi=300, bbox_inches='tight')
        plt.close()
        
        print("✓ Created epidemiological dashboard visualization")
    
    def _save_epidemiological_results(self, output_path, herd_immunity, risk_assessment,
                                    policy_scenarios, cost_benefit, vulnerability, predictions):
        """Save all epidemiological analysis results"""
        
        # Save policy recommendations
        recommendations = [
            "1. TARGETED INTERVENTIONS: Focus on regions with Very High and High risk levels",
            "2. SCHOOL ENTRY REQUIREMENTS: Implement and enforce vaccination requirements",
            "3. CATCH-UP CAMPAIGNS: Target adolescents and young adults with missed doses",
            "4. PUBLIC AWARENESS: Educate about herd immunity and disease risks",
            "5. HEALTHCARE WORKER TRAINING: Improve vaccine recommendation and delivery",
            "6. REMINDER SYSTEMS: Implement automated recall for second doses",
            "7. COMMUNITY ENGAGEMENT: Work with trusted community leaders",
            "8. DATA MONITORING: Real-time coverage monitoring with early warning systems"
        ]
        
        with open(output_path / 'policy_recommendations.txt', 'w') as f:
            f.write("MMR VACCINATION POLICY RECOMMENDATIONS\n")
            f.write("=" * 50 + "\n\n")
            for rec in recommendations:
                f.write(rec + "\n")
        
        # Save detailed analysis
        analysis_report = f"""
EPIDEMIOLOGICAL ANALYSIS REPORT
Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}

SUMMARY:
- Current effective MMR coverage: {herd_immunity['effective_coverage']:.1%}
- Gap to measles herd immunity: {herd_immunity['gaps']['measles']:.1%}
- Number of high-risk regions: {len(risk_assessment[risk_assessment['risk_level'].isin(['Very High Risk', 'High Risk'])])}
- Best policy scenario: {policy_scenarios.loc[policy_scenarios['Cases_Prevented_Pct'].idxmax(), 'Scenario']}
- Benefit-cost ratio: {cost_benefit['benefit_cost_ratio']:.2f}

CRITICAL FINDINGS:
1. Herd immunity for measles is currently NOT achieved
2. Several regions remain at high outbreak risk
3. Policy interventions can significantly reduce outbreak probability
4. Vaccination programs are cost-effective

URGENT ACTIONS NEEDED:
- Immediate targeted campaigns in high-risk regions
- Strengthening of school entry requirements
- Public health messaging about herd immunity
        """
        
        with open(output_path / 'epidemiological_analysis_report.txt', 'w') as f:
            f.write(analysis_report)
        
        print("✓ Saved policy recommendations and analysis report")

def clean_path_input(user_input):
    """Clean and validate user path input"""
    cleaned = user_input.strip().strip('"').strip("'").strip()
    if cleaned.startswith('r'):
        cleaned = cleaned[1:].strip().strip('"').strip("'")
    return cleaned

def main():
    print("=== MMR EPIDEMIOLOGICAL MODELLING FOR POLICY ===")
    print("=" * 60)
    
    try:
        # Get folder path from user
        user_input = input("Enter the path to your data folder (or press Enter for 'Data files'): ").strip()
        
        if not user_input:
            data_folder = "Data files"
        else:
            data_folder = clean_path_input(user_input)
        
        print(f"Looking for data in: {data_folder}")
        
        # Initialize model
        model = MMREpidemiologicalModel(data_folder_path=data_folder)
        
        # Load data
        model.load_data()
        
        # Create comprehensive epidemiological dashboard
        model.create_epidemiological_dashboard()
        
        print("\n" + "=" * 60)
        print("✅ EPIDEMIOLOGICAL MODELLING COMPLETE")
        print("📊 Check the 'Epidemiological_Models' folder for:")
        print("   - Epidemiological dashboard visualization")
        print("   - Policy recommendations")
        print("   - Detailed analysis report")
        print("   - Risk assessments and predictions")
        
    except FileNotFoundError as e:
        print(f"❌ ERROR: {e}")
        print("\nPlease check the data folder path and try again.")
        sys.exit(1)
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        import traceback
        traceback.print_exc()
        sys.exit(1)

if __name__ == "__main__":
    main()

=== MMR EPIDEMIOLOGICAL MODELLING FOR POLICY ===


Enter the path to your data folder (or press Enter for 'Data files'):  "C:\Users\GULLYHUB\Desktop\Data files"


Looking for data in: C:\Users\GULLYHUB\Desktop\Data files
Loading CSV data from 'C:\Users\GULLYHUB\Desktop\Data files'...
✅ Loaded 4320 MMR coverage records

📈 CREATING EPIDEMIOLOGICAL DASHBOARD

🛡️ HERD IMMUNITY ANALYSIS
Latest Year: 2019
Average Dose 2 Coverage: 88.6%
Effective Coverage (95% efficacy): 84.2%

Herd Immunity Thresholds:
Measles (R0=12): 91.7%
Mumps (R0=10): 90.0%
Rubella (R0=6): 83.3%

📊 Coverage Gaps to Herd Immunity:
Measles: 7.5%
Mumps: 5.8%
Rubella: 0.0%

⚠️ OUTBREAK RISK ASSESSMENT
Regional Outbreak Risk Assessment:
                  region  coverage  effective_coverage     risk_level  measles_susceptible
              north east  0.919656            0.873673      High Risk             0.046902
              north west  0.896882            0.852038      High Risk             0.070504
yorkshire and the humber  0.905918            0.860622      High Risk             0.061139
           east midlands  0.895495            0.850720      High Risk             0.071942
 